In [600]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('seaborn')

excel_name = '附件1 监测点A空气质量预报基础数据1.xlsx'#附件1 监测点A空气质量预报基础数据.xlsx
na_values = '—'

def cp(value, value_name):
    print('==============%s=============='%value_name)
    print(value)    
    print('==============%s=============='%value_name)

data1 = pd.read_excel(excel_name, sheet_name=0,na_values=na_values)
print('天数', len(data1)/72)
data1.head(3)

天数 356.0


,模型运行日期,预测时间,地点,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,...,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）,SO2小时平均浓度(μg/m³),NO2小时平均浓度(μg/m³),PM10小时平均浓度(μg/m³),PM2.5小时平均浓度(μg/m³),O3小时平均浓度(μg/m³),CO小时平均浓度(mg/m³)
0,2020-07-23,2020-07-23 00:00:00,监测点A,29.8890,304.016,0.018870,66.7409,4.16382,162.577,0.0,...,0.94818,428.278,0.0,0.0,2.40151,20.9208,8.17336,5.27729,8.78723,0.124491
1,2020-07-23,2020-07-23 01:00:00,监测点A,29.8736,303.739,0.017556,62.1551,4.65267,171.978,0.0,...,1.14987,427.531,0.0,0.0,1.44834,14.8144,6.49054,4.33106,12.74530,0.109056
2,2020-07-23,2020-07-23 02:00:00,监测点A,29.6471,303.419,0.017874,64.1760,4.10031,172.013,0.0,...,1.01616,427.428,0.0,0.0,1.27161,13.9154,6.86679,4.40045,12.22960,0.105957


In [601]:
data2 = pd.read_excel(excel_name, sheet_name=1, na_values=na_values)
print('天数', len(data2)/24)
data2.head(3)

天数 809.6666666666666


,监测时间,地点,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³),温度(℃),湿度(%),气压(MBar),风速(m/s),风向(°)
0,2019-04-16 00:00:00,监测点A,5.0,75.0,61.0,38.0,1.0,1.0,21.1,89.0,1013.0,1.2,346.5
1,2019-04-16 01:00:00,监测点A,5.0,64.0,60.0,41.0,25.0,0.8,20.4,92.0,1012.4,1.1,18.0
2,2019-04-16 02:00:00,监测点A,4.0,60.0,36.0,40.0,25.0,0.8,20.0,93.0,1011.3,1.1,65.9


In [602]:
data3 = pd.read_excel(excel_name, sheet_name=2, na_values=na_values)
data3.head(3)

,监测日期,地点,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3最大八小时滑动平均监测浓度(μg/m³),CO监测浓度(mg/m³)
0,2019-04-16,监测点A,5.0,56.0,36.0,31.0,103.0,0.8
1,2019-04-17,监测点A,9.0,63.0,67.0,53.0,206.0,1.1
2,2019-04-18,监测点A,6.0,37.0,31.0,25.0,83.0,0.6


# 数据清洗-缺失行补齐算法，缺失值，异常值

## 一次预测值的缺失预测天数的寻找(整个缺失，无法差值，因为定位删除)

In [603]:
start_time = '2020-07-23 00:00:00'
end_time = '2021-07-13 00:00:00'
time_list = pd.date_range(start_time, end_time, freq='D')
print('应有天数',len(time_list))
print('实际天数', len(data1)/72)

应有天数 356
实际天数 356.0


In [604]:
lack_list = list(set(time_list)-set(data1['模型运行日期']))
lack_list

[Timestamp('2021-05-21 00:00:00', freq='D'),
 Timestamp('2021-01-25 00:00:00', freq='D'),
 Timestamp('2020-11-11 00:00:00', freq='D')]

## 缺失行补齐算法(不补齐会造成很严重的后果)

In [605]:
start_time = '2019-04-16 00:00:00'
end_time = '2021-07-13 07:00:00'
time_list = pd.date_range(start_time, end_time, freq='H')
print('应有数量',len(time_list))
print('缺失行数量', len(time_list)-len(data2))
print('缺失行比例', 1-len(data2)/len(time_list))

应有数量 19664
缺失行数量 232
缺失行比例 0.01179820992676972


In [606]:
temp_df = data2.copy()
lack_list = list(set(time_list)-set(data2['监测时间']))
lack_df = pd.DataFrame(lack_list,columns=['监测时间'])
temp_df = pd.concat([temp_df,lack_df])
temp_df.sort_values("监测时间",inplace=True)
temp_df.index = list(range(len(temp_df)))
temp_df.shape

(19664, 13)

In [607]:
temp_df = temp_df
temp_df.iloc[:,2:] = temp_df.iloc[:,2:].interpolate()
temp_df.isnull().sum()

监测时间                  0
地点                  232
SO2监测浓度(μg/m³)        0
NO2监测浓度(μg/m³)        0
PM10监测浓度(μg/m³)       0
PM2.5监测浓度(μg/m³)      0
O3监测浓度(μg/m³)         0
CO监测浓度(mg/m³)         0
温度(℃)                 0
湿度(%)                 0
气压(MBar)              0
风速(m/s)               0
风向(°)                 0
dtype: int64

In [608]:
temp_df = temp_df.applymap(lambda v: max(0,v) if isinstance(v, float) else v)
temp_df.describe()

,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³),温度(℃),湿度(%),气压(MBar),风速(m/s),风向(°)
count,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000,19664.000000
mean,7.002085,32.882552,44.031911,23.588690,54.046125,0.716413,25.013578,69.302075,1010.588047,1.380802,154.056957
std,3.705630,23.865702,27.413365,18.387592,50.858145,0.238045,5.723049,16.128828,6.480650,0.669654,117.411160
min,0.000000,2.000000,0.000000,0.000000,0.000000,0.100000,5.800000,14.000000,993.500000,0.100000,0.000000
25%,5.000000,16.000000,24.000000,10.000000,17.000000,0.600000,21.000000,60.000000,1005.400000,0.900000,49.300000
50%,6.000000,27.000000,38.000000,19.000000,40.000000,0.700000,26.000000,71.000000,1010.000000,1.300000,108.000000
75%,9.000000,42.000000,58.000000,33.000000,76.000000,0.800000,29.400000,81.000000,1015.600000,1.800000,255.700000
max,47.000000,211.000000,217.000000,163.000000,405.000000,2.500000,38.200000,99.000000,1029.200000,5.800000,360.000000


## data1缺失值


In [609]:
data1.isnull().sum()

模型运行日期                0
预测时间                  0
地点                    0
近地2米温度（℃）             0
地表温度（K）               0
比湿（kg/kg）             0
湿度（%）                 0
近地10米风速（m/s）          0
近地10米风向（°）            0
雨量（mm）                0
云量                    0
边界层高度（m）              0
大气压（Kpa）              0
感热通量（W/m²）            0
潜热通量（W/m²）            0
长波辐射（W/m²）            0
短波辐射（W/m²）            0
地面太阳能辐射（W/m²）         0
SO2小时平均浓度(μg/m³)      0
NO2小时平均浓度(μg/m³)      0
PM10小时平均浓度(μg/m³)     0
PM2.5小时平均浓度(μg/m³)    0
O3小时平均浓度(μg/m³)       0
CO小时平均浓度(mg/m³)       0
dtype: int64

In [610]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   模型运行日期              25632 non-null  datetime64[ns]
 1   预测时间                25632 non-null  datetime64[ns]
 2   地点                  25632 non-null  object        
 3   近地2米温度（℃）           25632 non-null  float64       
 4   地表温度（K）             25632 non-null  float64       
 5   比湿（kg/kg）           25632 non-null  float64       
 6   湿度（%）               25632 non-null  float64       
 7   近地10米风速（m/s）        25632 non-null  float64       
 8   近地10米风向（°）          25632 non-null  float64       
 9   雨量（mm）              25632 non-null  float64       
 10  云量                  25632 non-null  float64       
 11  边界层高度（m）            25632 non-null  float64       
 12  大气压（Kpa）            25632 non-null  float64       
 13  感热通量（W/m²）          25632 non-null  float64   

# 数据准备-参数

## L为实测数据历史影响天数 

In [588]:
L=10
backward_length = 24*L
forward_length = 24*3
final_start_time = '2020-07-23 00:00:00'

for i in range(len(temp_df)):
    if temp_df['监测时间'][i+backward_length]!=pd.Timestamp(final_start_time):
        temp_df = temp_df.drop(labels=i) 
    else:
        break
temp_df.index=range(len(temp_df))
temp_df.head(3)

,监测时间,地点,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³),温度(℃),湿度(%),气压(MBar),风速(m/s),风向(°)
0,2020-07-13 00:00:00,监测点A,6.0,8.0,5.0,1.0,21.0,0.5,30.1,74.0,1006.6,2.0,235.8
1,2020-07-13 01:00:00,监测点A,6.0,8.0,7.0,2.0,21.0,0.5,29.8,76.0,1006.0,1.2,304.0
2,2020-07-13 02:00:00,监测点A,6.0,8.0,10.0,1.0,20.0,0.5,29.5,77.0,1005.6,1.1,19.7


In [589]:
temp_df.tail(3)

,监测时间,地点,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³),温度(℃),湿度(%),气压(MBar),风速(m/s),风向(°)
8765,2021-07-13 05:00:00,监测点A,9.0,19.0,9.0,2.0,8.0,0.4,29.2,78.0,1006.4,1.0,253.2
8766,2021-07-13 06:00:00,监测点A,8.0,20.0,22.0,5.0,6.0,0.4,29.0,81.0,1006.8,0.5,267.9
8767,2021-07-13 07:00:00,监测点A,8.0,14.0,22.0,5.0,19.0,0.4,30.2,75.0,1007.0,1.2,240.9


In [590]:
(len(temp_df)-7)/24

365.0416666666667

# 归一化

In [591]:
from sklearn.preprocessing import MinMaxScaler
#minmaxscaler
mm_channel1= MinMaxScaler()
mm_channel2= MinMaxScaler()
mm_channel3= MinMaxScaler()
mm_y = MinMaxScaler()


In [592]:
mm_y.fit_transform(temp_df.iloc[:,2:6+2])

array([[0.21428571, 0.02403846, 0.02304147, 0.00613497, 0.05185185,
        0.13043478],
       [0.21428571, 0.02403846, 0.03225806, 0.01226994, 0.05185185,
        0.13043478],
       [0.21428571, 0.02403846, 0.04608295, 0.00613497, 0.04938272,
        0.13043478],
       ...,
       [0.32142857, 0.07692308, 0.04147465, 0.01226994, 0.01975309,
        0.08695652],
       [0.28571429, 0.08173077, 0.10138249, 0.03067485, 0.01481481,
        0.08695652],
       [0.28571429, 0.05288462, 0.10138249, 0.03067485, 0.04691358,
        0.08695652]])

In [596]:
temp_df.iloc[:,2:] = mm_channel2.fit_transform(temp_df.iloc[:,2:])
temp_df

,监测时间,地点,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³),温度(℃),湿度(%),气压(MBar),风速(m/s),风向(°)
0,2020-07-13 00:00:00,监测点A,0.214286,0.024038,0.023041,0.006135,0.051852,0.130435,0.768987,0.714286,0.310976,0.333333,0.655000
1,2020-07-13 01:00:00,监测点A,0.214286,0.024038,0.032258,0.012270,0.051852,0.130435,0.759494,0.738095,0.292683,0.192982,0.844444
2,2020-07-13 02:00:00,监测点A,0.214286,0.024038,0.046083,0.006135,0.049383,0.130435,0.750000,0.750000,0.280488,0.175439,0.054722
3,2020-07-13 03:00:00,监测点A,0.214286,0.024038,0.064516,0.000000,0.046914,0.130435,0.750000,0.761905,0.277439,0.228070,0.724444
4,2020-07-13 04:00:00,监测点A,0.250000,0.028846,0.050691,0.006135,0.041975,0.130435,0.746835,0.761905,0.283537,0.122807,0.962222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8763,2021-07-13 03:00:00,监测点A,0.214286,0.043269,0.055300,0.000000,0.034568,0.086957,0.740506,0.761905,0.298780,0.122807,0.973611
8764,2021-07-13 04:00:00,监测点A,0.250000,0.057692,0.041475,0.000000,0.027160,0.086957,0.740506,0.761905,0.301829,0.105263,0.684167
8765,2021-07-13 05:00:00,监测点A,0.321429,0.076923,0.041475,0.012270,0.019753,0.086957,0.740506,0.761905,0.304878,0.157895,0.703333
8766,2021-07-13 06:00:00,监测点A,0.285714,0.081731,0.101382,0.030675,0.014815,0.086957,0.734177,0.797619,0.317073,0.070175,0.744167


In [611]:
data1.iloc[:,3:-6]

,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,云量,边界层高度（m）,大气压（Kpa）,感热通量（W/m²）,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）
0,29.8890,304.016,0.018870,66.7409,4.16382,162.577,0.000000,0.038201,769.903,100.665,20.1377,0.94818,428.278,0.0000,0.0000
1,29.8736,303.739,0.017556,62.1551,4.65267,171.978,0.000000,0.058892,682.085,100.671,15.8333,1.14987,427.531,0.0000,0.0000
2,29.6471,303.419,0.017874,64.1760,4.10031,172.013,0.000000,0.065039,627.078,100.678,12.3363,1.01616,427.428,0.0000,0.0000
3,29.4555,303.419,0.018935,68.7958,2.44317,168.135,0.047224,1.000000,636.497,100.675,11.1215,1.89003,442.472,0.0000,0.0000
4,28.5189,302.987,0.019881,76.5791,2.57759,207.884,8.260020,0.935074,490.162,100.714,19.8524,6.53753,458.394,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25627,34.3963,319.300,0.017129,45.9006,3.08236,157.020,0.982237,0.319844,1435.430,100.523,275.8630,49.94120,451.479,567.4070,681.8110
25628,34.3593,315.920,0.017551,47.1484,4.45631,164.549,1.460130,0.478448,1459.330,100.544,222.5160,41.65230,452.011,377.4950,453.6080
25629,33.8754,312.489,0.016992,47.0484,4.99797,152.778,1.184550,0.114264,1366.630,100.620,147.1860,27.88200,449.925,178.6100,214.6230
25630,32.6011,308.693,0.016529,49.4718,6.08861,141.533,0.063191,0.316036,607.047,100.635,86.3191,15.04470,444.482,21.8066,26.2034


In [612]:
data1.iloc[:,3:-6] = mm_channel1.fit_transform(data1.iloc[:,3:-6])
data1

,模型运行日期,预测时间,地点,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,...,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）,SO2小时平均浓度(μg/m³),NO2小时平均浓度(μg/m³),PM10小时平均浓度(μg/m³),PM2.5小时平均浓度(μg/m³),O3小时平均浓度(μg/m³),CO小时平均浓度(mg/m³)
0,2020-07-23,2020-07-23 00:00:00,监测点A,0.777364,0.588558,0.810240,0.642303,0.326912,0.451605,0.000000,...,0.038284,0.758434,0.000000,0.000000,2.401510,20.9208,8.17336,5.27729,8.787230,0.124491
1,2020-07-23,2020-07-23 01:00:00,监测点A,0.776912,0.582851,0.751803,0.591689,0.365404,0.477720,0.000000,...,0.040825,0.755567,0.000000,0.000000,1.448340,14.8144,6.49054,4.33106,12.745300,0.109056
2,2020-07-23,2020-07-23 02:00:00,监测点A,0.770262,0.576258,0.765924,0.613994,0.321911,0.477817,0.000000,...,0.039140,0.755172,0.000000,0.000000,1.271610,13.9154,6.86679,4.40045,12.229600,0.105957
3,2020-07-23,2020-07-23 03:00:00,监测点A,0.764636,0.576258,0.813125,0.664984,0.191429,0.467045,0.000548,...,0.050153,0.812912,0.000000,0.000000,0.467429,11.1535,5.25900,3.35261,13.780000,0.101764
4,2020-07-23,2020-07-23 04:00:00,监测点A,0.737135,0.567358,0.855182,0.750889,0.202013,0.577461,0.095821,...,0.108724,0.874022,0.000000,0.000000,0.574856,13.9989,6.05979,3.59303,9.963330,0.104536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25627,2021-07-13,2021-07-15 19:00:00,监测点A,0.909708,0.903438,0.732796,0.412285,0.241758,0.436169,0.011394,...,0.655727,0.847482,0.638242,0.638244,2.679800,44.8120,8.92721,5.66833,0.333226,0.224216
25628,2021-07-13,2021-07-15 20:00:00,监测点A,0.908622,0.833804,0.751563,0.426057,0.349943,0.457083,0.016938,...,0.551265,0.849524,0.424622,0.424623,2.937120,41.3621,9.00740,5.67797,0.007336,0.233528
25629,2021-07-13,2021-07-15 21:00:00,监测点A,0.894414,0.763118,0.726722,0.424954,0.392593,0.424385,0.013741,...,0.377722,0.841517,0.200908,0.200909,3.084680,41.5508,9.13525,5.76356,0.024794,0.237719
25630,2021-07-13,2021-07-15 22:00:00,监测点A,0.856997,0.684913,0.706109,0.451701,0.478469,0.393149,0.000733,...,0.215938,0.820627,0.024529,0.024529,4.003400,42.4011,8.24599,5.43220,0.000928,0.260096


In [613]:
data1.iloc[:,-6:] = mm_channel1.fit_transform(data1.iloc[:,-6:])
data1

,模型运行日期,预测时间,地点,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,...,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）,SO2小时平均浓度(μg/m³),NO2小时平均浓度(μg/m³),PM10小时平均浓度(μg/m³),PM2.5小时平均浓度(μg/m³),O3小时平均浓度(μg/m³),CO小时平均浓度(mg/m³)
0,2020-07-23,2020-07-23 00:00:00,监测点A,0.777364,0.588558,0.810240,0.642303,0.326912,0.451605,0.000000,...,0.038284,0.758434,0.000000,0.000000,0.017774,0.021943,0.030132,0.025552,0.021165,0.011173
1,2020-07-23,2020-07-23 01:00:00,监测点A,0.776912,0.582851,0.751803,0.591689,0.365404,0.477720,0.000000,...,0.040825,0.755567,0.000000,0.000000,0.010710,0.011819,0.023490,0.020584,0.030698,0.006234
2,2020-07-23,2020-07-23 02:00:00,监测点A,0.770262,0.576258,0.765924,0.613994,0.321911,0.477817,0.000000,...,0.039140,0.755172,0.000000,0.000000,0.009401,0.010328,0.024975,0.020948,0.029456,0.005242
3,2020-07-23,2020-07-23 03:00:00,监测点A,0.764636,0.576258,0.813125,0.664984,0.191429,0.467045,0.000548,...,0.050153,0.812912,0.000000,0.000000,0.003441,0.005749,0.018630,0.015447,0.033190,0.003900
4,2020-07-23,2020-07-23 04:00:00,监测点A,0.737135,0.567358,0.855182,0.750889,0.202013,0.577461,0.095821,...,0.108724,0.874022,0.000000,0.000000,0.004238,0.010467,0.021790,0.016709,0.023998,0.004787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25627,2021-07-13,2021-07-15 19:00:00,监测点A,0.909708,0.903438,0.732796,0.412285,0.241758,0.436169,0.011394,...,0.655727,0.847482,0.638242,0.638244,0.019836,0.061553,0.033107,0.027605,0.000803,0.043088
25628,2021-07-13,2021-07-15 20:00:00,监测点A,0.908622,0.833804,0.751563,0.426057,0.349943,0.457083,0.016938,...,0.551265,0.849524,0.424622,0.424623,0.021743,0.055833,0.033424,0.027656,0.000018,0.046068
25629,2021-07-13,2021-07-15 21:00:00,监测点A,0.894414,0.763118,0.726722,0.424954,0.392593,0.424385,0.013741,...,0.377722,0.841517,0.200908,0.200909,0.022836,0.056146,0.033928,0.028105,0.000060,0.047410
25630,2021-07-13,2021-07-15 22:00:00,监测点A,0.856997,0.684913,0.706109,0.451701,0.478469,0.393149,0.000733,...,0.215938,0.820627,0.024529,0.024529,0.029645,0.057556,0.030419,0.026366,0.000002,0.054571


## 实测数据 划分为 channel2，标签y

In [614]:
x1 = []
y = []
for i in range(365-L-3+1):
    line = i*24
    x1_temp = temp_df.iloc[line:line+backward_length,2:]
    y_temp = temp_df.iloc[line+backward_length:line+backward_length+forward_length,2:2+6]
    x1.append(x1_temp)
    y.append(y_temp)
x1 = np.array(x1)
y = np.array(y)
cp(x1_temp.shape, 'x1_temp.shape')
cp(y_temp.shape, 'y_temp.shape')
cp(x1.shape, 'x1.shape')
cp(y.shape, 'y.shape')
y_temp

==============x1_temp.shape==============
(240, 11)
==============x1_temp.shape==============
==============y_temp.shape==============
(72, 6)
==============y_temp.shape==============
==============x1.shape==============
(353, 240, 11)
==============x1.shape==============
==============y.shape==============
(353, 72, 6)
==============y.shape==============


,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³)
8688,9.0,20.0,10.0,4.0,30.0,0.7
8689,9.0,20.0,12.0,2.0,31.0,0.7
8690,9.0,19.0,27.0,4.0,31.0,0.7
8691,10.0,20.0,21.0,6.0,29.0,0.7
8692,12.0,20.0,208.0,4.0,26.0,0.7
...,...,...,...,...,...,...
8755,15.0,68.0,94.0,60.0,133.0,0.9
8756,16.0,103.0,87.0,74.0,51.0,0.9
8757,15.0,116.0,95.0,66.0,20.0,1.0
8758,15.0,133.0,91.0,64.0,2.0,1.0


## 将y从72小时平均到三天

In [615]:
def squzze(matrix):
    return np.array([matrix[:24].mean(axis=0),matrix[24:48].mean(axis=0),matrix[48:72].mean(axis=0)])

y_day = np.array([squzze(inst) for inst in y])
y_day.shape

(353, 3, 6)

## channel1，channel3

In [616]:
data1.shape[0]/72

356.0

In [617]:
x2 = []
for i in range(356-3):
    x2_temp = data1.iloc[i*72:(i+1)*72,3:]
    x2.append(x2_temp)
x2 = np.array(x2)
cp(x2_temp.shape, 'x2_temp.shape')
cp(x2.shape, 'x2.shape')
x2_temp

==============x2_temp.shape==============
(72, 21)
==============x2_temp.shape==============
==============x2.shape==============
(353, 72, 21)
==============x2.shape==============


,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,云量,边界层高度（m）,大气压（Kpa）,...,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）,SO2小时平均浓度(μg/m³),NO2小时平均浓度(μg/m³),PM10小时平均浓度(μg/m³),PM2.5小时平均浓度(μg/m³),O3小时平均浓度(μg/m³),CO小时平均浓度(mg/m³)
25344,0.754990,0.574095,0.837806,0.704042,0.293011,0.534343,0.000000,0.289651,0.193618,0.337771,...,0.036715,0.745596,0.000000,0.000000,0.042574,0.052500,0.026932,0.022917,0.000016,0.070683
25345,0.742743,0.562702,0.860740,0.746581,0.310699,0.525868,0.000000,0.222267,0.166517,0.335930,...,0.034739,0.741017,0.000000,0.000000,0.023693,0.046500,0.023169,0.019094,0.005743,0.053024
25346,0.736900,0.554935,0.852897,0.749521,0.313037,0.540799,0.000000,0.129328,0.189724,0.335624,...,0.034425,0.735736,0.000000,0.000000,0.015623,0.034697,0.023407,0.018676,0.029788,0.031076
25347,0.733485,0.549022,0.838771,0.742095,0.262289,0.534985,0.000000,0.174820,0.154416,0.338384,...,0.033559,0.724878,0.000000,0.000000,0.018314,0.037688,0.027292,0.020832,0.034351,0.022366
25348,0.726148,0.540885,0.818545,0.735631,0.246524,0.526651,0.000000,0.000000,0.200102,0.347586,...,0.033066,0.708447,0.000000,0.000000,0.024702,0.050566,0.029716,0.021798,0.019438,0.023035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25411,0.882372,0.829704,0.759620,0.461192,0.479825,0.368893,0.012914,0.346907,0.452879,0.288694,...,0.658475,0.846020,0.603689,0.603692,0.024501,0.069284,0.040190,0.037122,0.000250,0.056043
25412,0.889824,0.801170,0.758210,0.451699,0.391779,0.407688,0.000133,0.138954,0.555507,0.289307,...,0.455445,0.845037,0.408566,0.408567,0.039188,0.064929,0.033540,0.031061,0.000013,0.061820
25413,0.867251,0.731865,0.772727,0.488626,0.446631,0.439077,0.000000,0.875174,0.477598,0.297282,...,0.286572,0.837046,0.195077,0.195077,0.064035,0.065583,0.030996,0.030062,0.000023,0.082334
25414,0.842707,0.668432,0.786217,0.529664,0.482983,0.387843,0.000000,0.284172,0.360745,0.304337,...,0.115890,0.820711,0.027449,0.027449,0.069599,0.062786,0.025884,0.025927,0.000024,0.078782


## 将channel3从72小时平均到三天

In [618]:
channel3 = x2[:,:,-6:]
channel1 = x2[:,:,:-6]
channel3 = np.array([squzze(inst) for inst in channel3])
channel2 = x1
channel1.shape,channel2.shape,channel3.shape,y_day.shape

((353, 72, 15), (353, 240, 11), (353, 3, 6), (353, 3, 6))

# 模型

In [619]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = '6'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
tensorflow_version = tf.__version__
gpu_available = tf.test.is_gpu_available()
print("tensorflow_version", tensorflow_version, "GPU available", gpu_available)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
tensorflow_version 2.2.0 GPU available True


2021-10-17 01:17:38.030630: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-10-17 01:17:38.060656: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2200000000 Hz
2021-10-17 01:17:38.063585: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f44ac000b20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-17 01:17:38.063625: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-17 01:17:38.068117: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-17 01:17:40.859186: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558bed82d850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-10-17 01:17:40.859259: I tensorflow/compi

In [620]:
from tensorflow.keras.layers import Bidirectional
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Dense, Embedding, Dropout, GRU, MaxPooling1D, Lambda, Concatenate, \
    Multiply, RepeatVector, Flatten, Activation, Permute, Conv1D
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

index_from = 1#
batch_size = 32#256
embedding_size = 100#100
rnn_units = 64
drop_rate = 0.1
num_epoch = 50
model_name_path = 'BiGRU_best_model.h5'

def modelDef():
    input_text = Input(shape=(MAX_WORDS, ))
    embeddings = Embedding(input_dim=vocab_size + index_from+1,#vocab size
                           output_dim=embedding_size,
                           mask_zero=True, input_length=MAX_WORDS)(input_text)
    bigru=Bidirectional(GRU(units=rnn_units, return_sequences=False))(embeddings)#双向GRU层
    dropout = Dropout(drop_rate)(bigru)
    Softmax = Dense(label_num, activation="softmax", use_bias=True)(dropout)
    model = Model(inputs=[input_text], outputs=[Softmax])
    model.compile(optimizer="adam", loss='categorical_crossentropy',
                  metrics=['accuracy','top_k_categorical_accuracy'])
    return model

myModel = modelDef()
print(myModel.summary())
es = EarlyStopping(monitor='val_loss', patience=10)
cp = ModelCheckpoint(filepath = model_name_path, monitor='val_loss', save_best_only=True)
history = myModel.fit(x= x_train[100:100+100],
                    y= y_train[100:100+100],
                    batch_size=batch_size,
                    epochs=num_epoch,
                    verbose=1,callbacks=[es, cp],
                    validation_split=0.1)
# history = myModel.fit(x= X_train,
#                     y= y_train,
#                     batch_size=batch_size,
#                     epochs=num_epoch,
#                     verbose=2,
#                     validation_split=0.1)
# myModel.save(model_name_path)

#迭代图像
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(loss))
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend(loc='upper right')
plt.title('Train and Val Loss')
plt.show()


NameError: name 'MAX_WORDS' is not defined

In [347]:
import math

def compute_AQI(a):
    #a=np.array([n_SO2,n_NO2,n_PM10,n_PM25,n_O3,n_CO])
    T=np.array([[0,50,150,475,800,1600,2100,2620],[0,40,80,180,280,565,750,940],[0,50,150,250,350,420,500,600],[0,35,75,115,150,250,350,500],[0,100,160,215,265,800,0,0],[0,2,4,14,24,36,48,60]])
    I=np.array([0,50,100,150,200,300,400,500])
    BP_LO = []
    BP_Hi=[]
    IAQI_LO=[]
    IAQI_Hi=[]
    IAQI=np.zeros((1,6))
    result=[]
    for j in range(6):
        for i in range(1,9):
            if a[j] <= T[j, i]:
                BP_LO.append(T[j, i-1])
                BP_Hi.append(T[j, i])
                IAQI_LO.append(I[i-1])
                IAQI_Hi.append(I[i])
                break
            else:
                continue
        IAQI[0, j] = (IAQI_Hi[j] - IAQI_LO[j]) / (BP_Hi[j] - BP_LO[j]) * (a[j] - BP_LO[j]) + IAQI_LO[j]
        result.append(math.ceil(IAQI[0, j]))
    AQI=np.max(result)
    c=np.argmax(result)
    name=["SO2", "NO2", "PM10", "PM25", "O3", "CO"]
#     if AQI<=50:
#         print("AQI为%d" %(AQI))
#         print("当天无首要污染物")
#     else:
#         print("AQI为%d" % (AQI))
#         print('首要污染物为%s' % (name[c]))
    return AQI
    

AQI = [compute_AQI(inst) for inst in data2.iloc[:,2:8].values]
data2['AQI'] = AQI
fkdeplot(data2.iloc[:,-1].values, 'AQI')


IndexError: index 8 is out of bounds for axis 1 with size 8